In [1]:
import  requests
import  pandas as pd
import json

In [2]:
url="https://city.imd.gov.in/yogi/php/chart_min_max.php"
payload={
"INDEX_NUMBER":43063,
"DATE1":"2018-01-01",
"DATE2":"2024-10-05"
}

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',

}

response = requests.post(url, data=payload, headers=headers, verify=False)
response.encoding = 'utf-8'
# Check the status code


C:\Users\shind\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'city.imd.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [6]:
if response.status_code == 200:
    print('Request successful!')
    print(type(response.text))
    data = json.loads(response.text)
    filt_data=data['data_max'][1:]
    #print(filt_data)
    df1 = pd.json_normalize(filt_data)
    #print(data['data_max'])
    
    filt_data2=data['data_min'][1:]
    #print(filt_data)
    df2 = pd.json_normalize(filt_data2)
    #print(df1.shape)
    print (df1, df2)
else:
    print(f'Request failed with status code: {response.status_code}')

Request successful!
<class 'str'>
         Datee Parameter1 Parameter2
0     01-01-22       28.8       28.9
1     02-01-22       29.2       28.9
2     03-01-22       29.7       28.9
3     04-01-22       29.4       28.9
4     05-01-22       30.7       28.9
...        ...        ...        ...
1004  01-10-24       34.9       30.9
1005  02-10-24       33.8       30.9
1006  03-10-24       31.9       31.3
1007  04-10-24       31.9       31.3
1008  05-10-24       32.9       31.3

[1009 rows x 3 columns]          Datee Parameter1 Parameter2
0     01-01-22       13.4       11.3
1     02-01-22       15.7       11.3
2     03-01-22       14.5       11.3
3     04-01-22       14.2       11.3
4     05-01-22       13.5       11.3
...        ...        ...        ...
1004  01-10-24       21.9       21.0
1005  02-10-24       23.4       21.0
1006  03-10-24       23.2       20.6
1007  04-10-24       21.5       20.6
1008  05-10-24       20.2       20.6

[1009 rows x 3 columns]


In [7]:
df1.rename(columns={'Datee': 'date', 'Parameter1': 'Tmax','Parameter2': 'Normal of Tmax'}, inplace=True)
df2.rename(columns={'Datee': 'date', 'Parameter1': 'Tmin','Parameter2': 'Normal of Tmin'}, inplace=True)

In [8]:
merged_df = pd.merge(df1, df2, on='date', how='inner')
merged_df

,date,Tmax,Normal of Tmax,Tmin,Normal of Tmin
0,01-01-22,28.8,28.9,13.4,11.3
1,02-01-22,29.2,28.9,15.7,11.3
2,03-01-22,29.7,28.9,14.5,11.3
3,04-01-22,29.4,28.9,14.2,11.3
4,05-01-22,30.7,28.9,13.5,11.3
...,...,...,...,...,...
1004,01-10-24,34.9,30.9,21.9,21.0
1005,02-10-24,33.8,30.9,23.4,21.0
1006,03-10-24,31.9,31.3,23.2,20.6
1007,04-10-24,31.9,31.3,21.5,20.6


In [9]:
#null data printing
null_mask = merged_df.isnull()
print("\nNull Mask:")
#print(null_mask)
print(merged_df[merged_df.isnull().any(axis=1)])


Null Mask:
         date  Tmax Normal of Tmax  Tmin Normal of Tmin
250  08-09-22  None           29.0  21.6           21.0
451  28-03-23  None           37.2  14.3           17.8
529  14-06-23  None           32.4  25.4           23.3
601  25-08-23  None           28.2  21.9           21.4
666  29-10-23  None           31.5  14.5           15.9
718  20-12-23  None           29.2  12.0           11.0
904  23-06-24  None           30.4  23.6           23.0
958  16-08-24  None           27.6  22.2           21.8


In [10]:
merged_df_cleaned = merged_df.dropna()
merged_df_cleaned

,date,Tmax,Normal of Tmax,Tmin,Normal of Tmin
0,01-01-22,28.8,28.9,13.4,11.3
1,02-01-22,29.2,28.9,15.7,11.3
2,03-01-22,29.7,28.9,14.5,11.3
3,04-01-22,29.4,28.9,14.2,11.3
4,05-01-22,30.7,28.9,13.5,11.3
...,...,...,...,...,...
1004,01-10-24,34.9,30.9,21.9,21.0
1005,02-10-24,33.8,30.9,23.4,21.0
1006,03-10-24,31.9,31.3,23.2,20.6
1007,04-10-24,31.9,31.3,21.5,20.6


In [11]:
merged_df_cleaned.to_csv('new-pune-shivajinagar-{}-to-{}.csv'.format(payload['DATE1'],payload['DATE2']))

In [12]:
#reading as test
new_fd=pd.read_csv("./new-pune-shivajinagar-2018-01-01-to-2024-10-05.csv")

In [13]:
new_fd

,Unnamed: 0,date,Tmax,Normal of Tmax,Tmin,Normal of Tmin
0,0,01-01-22,28.8,28.9,13.4,11.3
1,1,02-01-22,29.2,28.9,15.7,11.3
2,2,03-01-22,29.7,28.9,14.5,11.3
3,3,04-01-22,29.4,28.9,14.2,11.3
4,4,05-01-22,30.7,28.9,13.5,11.3
...,...,...,...,...,...,...
996,1004,01-10-24,34.9,30.9,21.9,21.0
997,1005,02-10-24,33.8,30.9,23.4,21.0
998,1006,03-10-24,31.9,31.3,23.2,20.6
999,1007,04-10-24,31.9,31.3,21.5,20.6
